In [ ]:
# pip install requests
import requests
from bs4 import BeautifulSoup


url = "https://mykbostats.com/players/1"
apikey = "b327144420a12f944bbdd4863c9fc0bffb28a1e3" 
params = {
    "url": url,
    "apikey": apikey,
    "js_render": "true",
    "premium_proxy": "true",
}
response = requests.get("https://api.zenrows.com/v1/", params=params)

#print(response.content[:500])

soup = BeautifulSoup(response.content, 'html.parser')

In [ ]:
# extract table header information
header = soup.select('thead th')

for i in range(len(header)):
  header[i] = header[i].text

# remove 'Game Stats' data
header = header[:header.index("Date")]



In [ ]:
# did not test for case when page is empty ex. https://mykbostats.com/players/2000

rows = soup.select('tbody tr')

r = rows[0]


print(r.select_one('.left').text) #use .strip()?
print(r.select_one('a').text)
print('\n----------------------------------\n')
print(r)


#for i in rows: print(i.text)


#for i in range(len(rows)): 
  

2021
NC

----------------------------------

<tr>
<td class="stick left">2021</td>
<td>
<nobr><span class="team-logo nc"></span>
<a href="/teams/9">NC</a></nobr>
</td>
<td>7.83</td>
<td>1.70</td>
<td>3</td>
<td>2</td>
<td>0</td>
<td>1</td>
<td>0</td>
<td>12</td>
<td>3</td>
<td>0 (0)</td>
<td>0</td>
<td>109</td>
<td>435</td>
<td>23</td>
<td>21</td>
<td>20</td>
<td>25</td>
<td>4</td>
<td>1</td>
<td>4</td>
<td>14</td>
<td>14</td>
<td>0</td>
<td>0</td>
<td>3</td>
<td>0</td>
</tr>
